# Data Exploration using Snowflake's Native Pandas -- Modin

This notebook will explore a dataset loaded into Snowflake, leveraging **Snowpark "Modin" Dataframes** to natively handle large datasets efficiently and directly within the Snowflake environment. This keeps all operations scalable and optimized for Snowflake's architecture while enabling distributed processing. We will also perform standard feature engineering tasks.

### Key Objectives:
- Inspect the dataset (schema, missing values, basic statistics)
- Explore numerical and categorical features
- Visualize a subset of the data

## 0 - Import Packages and Get Active Snowflake Session Object

In [ ]:
# Import required libraries
import numpy as np

import snowflake.snowpark.functions as F
from snowflake.snowpark.context import get_active_session
import modin.pandas as spd  # Use Modin's Snowpark Pandas version
import snowflake.snowpark.modin.plugin  # Plugin to connect Modin with Snowflake

In [ ]:
## Session Initialization
#   Snowflake Notebooks automatically manage sessions, so we don't need to set up the connection manually.
#   We'll retrieve the active session using 'get_active_session()'.
#   We are using an x-small warehouse for this demo.

session = get_active_session()

## 1 - Load Data Into Snowflake for Exploration


### 1.1 Load Data into Snowflake Pandas Dataframes

Loading Data into Snowflake using Snowflake Modin Pandas Plugin
In this notebook, we will use the **Snowflake Snowpark Modin Pandas plugin** to load large CSV files into Snowflake. This plugin allows efficient data loading and processing in parallel using Modin.

In [ ]:
# Load the CSV files using Snowpark's Modin Pandas plugin
application_record_df = spd.read_csv('data/application_record.csv.zip')
credit_record_df = spd.read_csv('data/credit_record.csv.zip')

In [ ]:
# Display the first few rows
application_record_df.describe()

In [ ]:
# Display the last few rows
credit_record_df.describe()

### 1.2 Upload Data to Snowflake

Now that the data is loaded, we can upload it directly to Snowflake using the `write_pandas` method.
This creates the necessary tables and uploads the data to Snowflake.

In [ ]:
# Upload the DataFrames to Snowflake tables
session.write_pandas(application_record_df, table_name='APPLICATION_RECORD', auto_create_table=True, overwrite=True)
session.write_pandas(credit_record_df, table_name='CREDIT_RECORD', auto_create_table=True, overwrite=True)

## 2 - Data Structure Exploration

We will begin with some basic transformations and analysis on 'application_record_df'.

### 2.1 - Basic Transformations

1. Convert age from days to years
2. Find average age per gender and income type.

In [ ]:
# Convert DAYS_BIRTH to numeric (if needed)
application_record_df['DAYS_BIRTH'] = spd.to_numeric(application_record_df['DAYS_BIRTH'], errors='coerce')

# Now create the AGE column and ensure it's an integer
application_record_df['AGE'] = application_record_df['DAYS_BIRTH'].apply(lambda x: np.floor(abs(x) / 365)).astype(int)

# Display the first few rows to verify
print(application_record_df[['DAYS_BIRTH', 'AGE']].head())

### 2.2 - Data Exploration

1. Count Total Rows
2. Drop Duplicates
3. Get Summary Stats for Numerical Columns
4. Schema Exploration

In [ ]:
## 1. Count Total Rows
# Count the number of rows in application_record_df
row_count = application_record_df.count()

print(f"Total number of rows: {row_count}")

In [ ]:
## 2. Drop Duplicates
# Drop duplicates based on the 'ID' column
application_record_df = application_record_df.drop_duplicates('ID')

print(f"Number of rows after dropping duplicates: {application_record_df.count()}")

In [ ]:
## 3. Get Summary Stats for Numerical Columns
# We can generate descriptive statistics for numerical columns to understand the data distribution.

# Identify numerical columns (integer and float types)
numerical_columns = application_record_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"Numerical columns: \n {numerical_columns}")

In [ ]:
## 4. Schema Exploration
# We start by inspecting the schema to identify the data types of each column. 
# This exploration involves schema inspection, identifying categorical and numerical columns, and summarizing the distribution of income by gender and income type.

# Show the schema of the DataFrame
print(application_record_df.dtypes)

In [ ]:
# Categorical and Numerical Features
# Now, let’s identify which columns are categorical and which are numerical. 

# Identify categorical columns (string types)
categorical_columns = application_record_df.select_dtypes(include='object').columns.tolist()
print(f"Categorical columns: \n {categorical_columns}")

In [ ]:
# Unique Values in Categorical Columns
# To understand the diversity in the categorical data, we will compute the number of unique values in each categorical column.
# Calculate the number of unique values per categorical column
unique_values = [(col, application_record_df[col].nunique()) for col in categorical_columns]

# Convert to DataFrame and display the results
unique_values_df = spd.DataFrame(unique_values, columns=['COLUMN_NAME', 'NUM_UNIQUE_VALUES'])
print(unique_values_df)

In [ ]:

# Distribution of Income by Income Type
# Next, we will compute the average income grouped by NAME_INCOME_TYPE. 
# Using groupby and agg, this operation is distributed and faster for large datasets.

# Group by 'NAME_INCOME_TYPE' and 'CODE_GENDER', calculate the average income
analysis_df = application_record_df.groupby('NAME_INCOME_TYPE').agg({'AMT_INCOME_TOTAL': 'mean'}).reset_index()

# Rename the column for better readability
analysis_df.columns = ['NAME_INCOME_TYPE', 'AVG_INCOME']

# Sort the results by 'NAME_INCOME_TYPE' (ascending) and 'AVG_INCOME' (descending)
analysis_df = analysis_df.sort_values(by=['NAME_INCOME_TYPE', 'AVG_INCOME'], ascending=[True, False])


# Display the final result!
print(analysis_df)

## 3 - Persist Transformations
If we want to save the changes we can either save it as a table, meaning the SQL generated by the DataFrame is executed and the result is stored in a table or as a view where the DataFrame SQL will be the definition of the view.
save_as_table saves the result in a table, if mode='overwrite' then it will also replace the data that is in it.

In [ ]:
session.write_pandas(application_record_df, table_name='APPLICATION_RECORD', auto_create_table=True, overwrite=True)
session.table('APPLICATION_RECORD').show()

In [ ]:
select *
from application_record
fetch 5;

In [ ]:
session.close()

# Conclusion

The goal of this notebook is to help you understand how to use Pandas natively in Snowflake Snowpark. By integrating Pandas with Snowpark, you can leverage powerful data manipulation and analysis tools directly within your Snowflake environment, enhancing your data workflows and efficiency.

If you are interested in more guides, please consider:

- Following me on [LinkedIn](https://www.linkedin.com/in/mattstrautmann/) for updates, tutorials, and professional insights.
- Subscribing to my [Snowbits AI Kickstart blog](https://mattstrautmann.substack.com/?utm_source=github&utm_medium=web&utm_campaign=substack_profile) for in-depth articles and the latest trends in data analytics and AI.



*Thank you for exploring this notebook!*